# Main notebook for project 2

Main script is ``model.py`` but this notebook can be used for practice etc.

In [14]:
import torch
import math
import time
import datetime
import random

torch.set_grad_enabled(False)

## Module

In [15]:
class Module(object):
    
    def __init__(self):
        pass
    
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError 
        
    def param(self):
        return []

## Sequential

In [16]:
class Sequential(object):
    
    def __init__(self):
        raise NotImplementedError
    
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError 
        
    def param(self):
        return []

## Layers

In [17]:
class Conv2d(Module):
    
    def __init__(self):
        raise NotImplementedError
    
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError 
        
    def param(self):
        return []

In [18]:
class NearestUpsampling(Module):
   
    def __init__(self):
        raise NotImplementedError
    
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError 
        
    def param(self):
        return []

## Activation functions

In [19]:
class ReLU(Module):
    
    def __init__(self):
        raise NotImplementedError
    
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError

In [20]:
class Sigmoid(Module):
    
    def __init__(self):
        raise NotImplementedError
    
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError

## Loss

In [21]:
class MSE(Module):
    
    def __init__(self):
        raise NotImplementedError
    
    def forward (self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError

## Optimizer

In [22]:
class SGD():
    
    def __init__(self):
        raise NotImplementedError
    
    def step(self):
        raise NotImplementedError
        
    def zero_grad(self):    
        raise NotImplementedError